## Master Jedi Notebook

### This notebook analyzes information about items, users, credits, etc from your AGO organization

#### Run this cell to connect to your GIS and get started:

In [ ]:
# Destination feature layers that will be update

# Configure this after they have been created
# pattern is "name": itemID
# such as "Example": "123456789abcdefghijklmnopqrstu"

outputHFLs = {"PubEditItems": "",
              "CreatedModifiedItems": "",
              "WeeklyCreditHistory": "",
              "AGO_LoginAnalysis": "",
              "CreditsRemaining": ""}

In [ ]:
from arcgis.gis import GIS

import pandas as pd

gis = GIS("home")

In [ ]:
# Let's grab all the information we can about all of the items in the organization to analyze

MaxSearch = 9999  # Change the maximum number of items returned for testing purposes

allItems = gis.content.search(query="", max_items=MaxSearch)

allItemsDF = pd.DataFrame(allItems)

auditLen = len(allItems)
if auditLen >= MaxSearch:
    print("The maximum number of records has been reached.")
print("Found {} items to asses".format(auditLen))

allItemsDF

## Write various results to a hosted, published *.csv

In [ ]:
from arcgis.features import FeatureLayerCollection
def Results2AGO(csvName, df):

    # Create a temporary *.csv
    import os, tempfile
    tempdir = tempfile.TemporaryDirectory(suffix=None, prefix=None, dir=None)
    print (tempdir)

    ### if executing manually, you can input the item name
    # csvName = input("What do you want to call the csv? ")

    csvpath = (os.path.join(tempdir.name, csvName + ".csv"))
    print (csvpath)
    df.to_csv(csvpath, index=False)

    #Find out if it is already in AGO
    prevCSV = gis.content.search(query=csvName, item_type="Feature Layer Collection", max_items = 1)
    if prevCSV:
        print("Found a previous csv in AGO")
        prevCSV = prevCSV[0]
        display(prevCSV)
       
        prevCSVcollection = FeatureLayerCollection.fromitem(prevCSV)
        print("The table already exists in AGO")

        # We will now replace the old data with new
        prevCSVcollection.manager.overwrite(csvpath)
        display

    else:
        print("We have to upload it")
        csv_item = gis.content.add({}, csvpath)
        display(csv_item)
        csv_lyr = csv_item.publish()
        display(csv_lyr)

    tempdir.cleanup()
    del csvName
    print ("The CSV has been published.\nAll done, thanks for playing!")

### Let's check if it is a service that allows public editing

In [ ]:
from arcgis.features import FeatureLayerCollection

PubEditing = []

for i in allItems:
    if i.access == 'public':
        if i.type == 'Feature Service':
            try:
                i_flc = FeatureLayerCollection.fromitem(i)
                if 'Editing' in i_flc.properties.capabilities:
                    PubEditing.append(i.title)
            except:
                print("Wasn't able to assess {}".format(i.title))
print(PubEditing)

if len(PubEditing) > 0:
    print("Found {} items with public editing allowed".format(len(PubEditing)))
    dict = {'Public Editing Items': PubEditing}  
    PEdf = pd.DataFrame(dict)
    
    # Now let's create/update a *.csv to hold this information
    print("Throwing it over to the csv writer/updater")
    Results2AGO(outputHFLs["PubEditItems"], PEdf)

### Let's find created and updated dates

In [ ]:
from datetime import datetime
import time
from requests.compat import urljoin

CreateDates = []
ModifiedDates =[]
hyperlinkField = []

# change your base arcgis organiztion url here
urlBase='https://dublinohio.maps.arcgis.com/home/item.html'

for item in allItems:
    CreateDate = datetime.fromtimestamp(item["created"]/1000) # Get the date created in human readable date format
    #print(item.title, "created on: {}".format(CreateDate))
    CreateDates.append(CreateDate)
    ModifiedDate = datetime.fromtimestamp(item["modified"]/1000)
    #print(item.title, "modified on: {}".format(ModifiedDate))
    ModifiedDates.append(ModifiedDate)
    
    urlQuery = '?id=' + item.id
    url_full = urljoin(urlBase,urlQuery)
    #print(item.title, "accessed here: {}".format(url_full))
    hyperlinkField.append(url_full)

print("Created {} created dates".format(len(CreateDates)))
print("Created {} modified dates".format(len(ModifiedDates)))
print("Created {} hyperlinks".format(len(hyperlinkField)))

#### Now that we have lists for all the dates and a hyperlink, let's add them to a new dataframe

# this creates a new dataframe from the original one, but with less columns
createdColumns = ['id','title','name','owner','type','size','created','modified',]
createdDF = allItemsDF[allItemsDF.columns.intersection(createdColumns)]

createdDF.insert(3, "createdDate", CreateDates, True) # adds and populates a created field as date
createdDF.insert(5, "modifiedDate", ModifiedDates, True) # adds and populates a modified field as date
createdDF['Hyperlink_field'] = hyperlinkField # adds a hyperlink field to the end

createdDF

In [ ]:
### Calls the hosted *.csv portion
### BE SURE TO RUN THAT FIRST

if len(createdDF) > 0:
    # Now let's create/update a *.csv to hold this information
    print("Throwing it over to the csv writer/updater")
    Results2AGO(outputHFLs["CreatedModifiedItems"], createdDF)

### Credits Categories

In [ ]:
### This gathers the credit usage by category for the past X time (for instance, 7 days)

cm = gis.admin.credits
from datetime import datetime
from datetime import timedelta
from arcgis.features import FeatureLayer
import pandas as pd

tDay = datetime.now().date()

# It is maybe silly to start with a date object, convert it into components and then reconstruct a date
# However, that is how it must be done, confirmed through a support ticket with Esri
lastWeek = tDay - timedelta(days=7) # this is where the time interval is set
thisDay = int(tDay.strftime("%d"))
thisMonth = int(tDay.strftime("%m"))
thisYr = int(tDay.strftime("%Y"))
lastDay = int(lastWeek.strftime("%d"))
lastMonth = int(lastWeek.strftime("%m"))
lastYr = int(lastWeek.strftime("%Y"))

start_date = datetime(lastYr, lastMonth, lastDay)
end_date7 = datetime(thisYr, thisMonth, thisDay)

print(start_date)
print(end_date7)

# This is where the actual credit usage is returned
credits7 = cm.credit_usage(start_time=end_date7, end_time=start_date)
print(credits7)

# Variables of the various credit usage types are assigned and checked for returns
DayEnding = tDay
applogin = credits7.get('applogin') if credits7.get('applogin') else 0
features = credits7.get('features')
geoenrich = credits7.get('geoenrich') if credits7.get('geoenrich') else 0
intnotebks = credits7.get('intnotebks')
notebooks = credits7.get('notebooks')
portal = credits7.get('portal')
schdnotebks = credits7.get('schdnotebks')
spanalysis = credits7.get('spanalysis')
tiles = credits7.get('tiles')
vectortiles = credits7.get('vectortiles')

# We will read in the existing data in order to retain it
# add in the url of the actual table like below...
table_url ="https://services1.arcgis.com/notreal/arcgis/rest/services/WeeklyCreditHistory/FeatureServer/0"
tbl = FeatureLayer(table_url, gis=gis)

oldWCHdf = tbl.query(where='1=1',as_df='true') #querying without any conditions returns all the features

# Now we will add a record for the new data to the already loaded prior data
newWCHdf = oldWCHdf.append({
'DayEnding':tDay,
'applogin':applogin,
'features':features,
'geoenrich':geoenrich,
'intnotebks':intnotebks,
'notebooks':notebooks,
'portal':portal,
'schdnotebks':schdnotebks,
'spanalysis':spanalysis,
'tiles':tiles,
'vectortiles':vectortiles,
'ObjectId':oldWCHdf["ObjectId"].max()+1
}, ignore_index=True)

newWCHdf

In [ ]:
### Calls the hosted *.csv portion
### BE SURE TO RUN THAT FIRST

if len(newWCHdf) > 0:
    # Now let's create/update a *.csv to hold this information
    print("Throwing it over to the csv writer/updater")
    
    Results2AGO(outputHFLs["WeeklyCreditHistory"], newWCHdf)

### AGO Logins

In [ ]:
### We can report on how people are logging into ArcGIS Online
### This is not only who is logging in, but also through what app
### This can be arcgis.com, QuickCapture, dashboards, etc.

from datetime import datetime, timedelta
import pandas as pd

# We need to set up the dates for which we're going to analyze
tDay = datetime.now().date()
lastWeek = tDay - timedelta(days=2)
thisDay = int(tDay.strftime("%d"))
thisMonth = int(tDay.strftime("%m"))
thisYr = int(tDay.strftime("%Y"))
lastDay = int(lastWeek.strftime("%d"))
lastMonth = int(lastWeek.strftime("%m"))
lastYr = int(lastWeek.strftime("%Y"))
# print(tDay)
# print(lastWeek)
# print(thisDay)
# print(thisMonth)
# print(thisYr)
# print(lastDay)
# print(lastMonth)
# print(lastYr)

start_date = datetime(thisYr, thisMonth, thisDay)
to_date = datetime(lastYr, lastMonth, lastDay)
print(to_date)
print(start_date)

# retreive the data
Agologins = gis.admin.history(to_date, start_date, num = 9999, data_format='df')

# drop columns for security and extraneous data
Agologins.drop(['id','idType','orgId','ip','reqId','owner','data'], axis = 1, inplace = True)

localTimes = []

import datetime
for index, row in Agologins.iterrows():
    #print(row['created'])
    localTime = datetime.datetime.fromtimestamp(int(row['created'])/1000)
    localTimes.append(localTime.strftime("%Y-%m-%d %H:%M:%S"))

Agologins['CreatedDate'] = localTimes

splitcolumnns = ['actor','appId']

AGOloginAnalysis = Agologins.groupby(splitcolumnns).agg(appIdCount = ("appId", "count"))

# reset index because of multi-level indexing from groupby made for weird things. 
AGOloginAnalysis.reset_index(inplace=True)
AGOloginAnalysis

In [ ]:
### Calls the hosted *.csv portion
### BE SURE TO RUN THAT FIRST

if len(AGOloginAnalysis) > 0:
    # Now let's create/update a *.csv to hold this information
    print("Throwing it over to the csv writer/updater")
    #csvName = 
    Results2AGO(outputHFLs['AGO_LoginAnalysis'], AGOloginAnalysis)

### Credits Remaining

In [ ]:
### This will return how many credits are left

from datetime import datetime, timedelta
tday = datetime.now()

creditsleft = gis.admin.credits.credits
print(creditsleft)

In [ ]:
### Now that we know how many credits are remaining
### we should add this to the hosted table

from arcgis.features import FeatureLayer

# We will read in the existing data in order to retain it
# add in the url of the actual table like below...
table_url = "https://services1.arcgis.com/notreal/arcgis/rest/services/CreditsRemaining/FeatureServer/0"
tbl = FeatureLayer(table_url, gis=gis)

import pandas as pd
oldCRdf = tbl.query(where='1=1',as_df='true') #querying without any conditions returns all the features
#oldCRdf

# Now we will add a record for the new data to the already loaded prior data
newCRdf = oldCRdf.append({
        'Tday':tday,
        'CreditsRemaining':creditsleft,
        'ObjectId':oldCRdf["ObjectId"].max()+1
        }, ignore_index=True)

newCRdf

In [ ]:
### Calls the hosted *.csv portion
### BE SURE TO RUN THAT FIRST

if len(newCRdf) > 0:
    # Now let's create/update a *.csv to hold this information
    print("Throwing it over to the csv writer/updater")
    #csvName = "AllItems"
    Results2AGO(outputHFLs['CreditsRemaining'], newCRdf)